In [ ]:
from dotenv import load_dotenv
import os
import json
import sys
from typing import Optional, Dict, Any, List

from openai import OpenAI

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    print("환경변수 OPENAI_API_KEY가 설정되어 있지 않습니다. .env 파일 또는 환경변수에 키를 넣어주세요.")
    sys.exit(1)

client = OpenAI(api_key=OPENAI_API_KEY)

FIELDS = [
    # 필수 수집 항목
    ("purpose", "선물 목적이 어떻게 되시나요?"),
    ("relation", "선물할 사람과의 관계는 무엇인가요?"),
    ("gender", "선물받는 분의 성별은 무엇인가요?"),
    ("preferred_color", "원하시는 색상을 입력해주세요."),
    ("personality", "식물을 키울 사람의 성향(성격)을 간단히 알려주세요."),
    ("has_pet", "강아지나 고양이를 키우나요?"),
    # 추가 수집 항목
    ("user_experience", "선물받는 분의 원예 경험이 있나요?"),
    ("preferred_style", "원하는 스타일이 있나요?"),
    ("plant_type", "원하는 식물 유형이 있나요?"),
    ("season", "선물할 시기(계절)가 정해져 있나요?"),
    ("humidity", "특별히 신경쓸 실내 습도가 있나요?"),
    ("isAirCond", "에어컨 사용이 잦은 환경인가요?"),
    ("watering_frequency", "물을 얼마나 자주 줄 수 있을 것 같나요?"),
    ("emotion", "선물에 담고 싶은 감정이나 메시지가 있나요?"),
]

# 맨 처음 질문
INITIAL_OPEN_QUESTION = (
    "선물하실 상황을 간단히 알려주세요. (예: 친구 졸업식 선물, 부모님 기념일 등) — "
    "누구에게, 어떤 목적, 언제 주실 예정인지 한두 문장으로 설명해 주세요."
)

# 모른다고 했을 떄
def is_unknown(answer: Optional[str]) -> bool:
    if answer is None:
        return True
    a = answer.strip().lower()
    if a == "":
        return True
    unknown_keywords = ["모름", "모르겠", "모르겠어요", "몰라", "모르겠음"]
    for kw in unknown_keywords:
        if kw in a:
            return True
    return False

def parse_pets(answer: Optional[str]) -> Dict[str, Optional[bool]]:
    if answer is None:
        return {"has_dog": None, "has_cat": None}
    a = answer.strip().lower()
    if is_unknown(a):
        return {"has_dog": None, "has_cat": None}
    has_dog = None
    has_cat = None
    if any(k in a for k in ["강아지", "개"]):
        has_dog = True
    if any(k in a for k in ["고양이", "고양"]):
        has_cat = True
    if any(k in a for k in ["없음", "없어", "안키움", "아니오", "없습니다"]):
        has_dog = False if has_dog is None else has_dog
        has_cat = False if has_cat is None else has_cat
    return {"has_dog": has_dog, "has_cat": has_cat}

# 전체 대화로부터 엄격한 JSON을 생성 (종결 시 사용)
def extract_json_from_conversation(conversation: List[Dict[str, str]]) -> Dict[str, Any]:
    system_prompt = (
        "당신은 친절한 식물 전문가입니다. 한국어로 친절하게 답변하세요.\n"
        "아래 형식을 반드시 지키되, 실제 상담사가 말하듯 자연스럽고 단정적인 말투로 작성하세요."
        "다음 키들을 포함하는 JSON 하나만 출력하세요. "
        "만약 사용자가 모른다고 했거나 관련 대화가 전혀 없다면 그 필드의 값은 null로 하세요. "
        "사용자가 'pass'로 넘긴 질문은 무시하되 해당 값이 채워지지 않았다면 null로 두세요. "
        "사용자가 강아지/고양이 관련 대답을 했으면 has_dog/has_cat 을 True/False/null 로 정확히 채우세요. "
        "출력 형식(예시):\n"
        "{\n"
        '  "purpose": "...",\n'
        '  "relation": "...",\n'
        '  "gender": "...",\n'
        '  "preferred_color": "...",\n'
        '  "personality": "...",\n'
        '  "has_dog": true/false/null,\n'
        '  "has_cat": true/false/null,\n'
        '  "user_experience": "...",\n'
        '  "preferred_style": "...",\n'
        '  "plant_type": "...",\n'
        '  "season": "...",\n'
        '  "humidity": "...",\n'
        '  "isAirCond": "...",\n'
        '  "watering_frequency": "...",\n'
        '  "emotion": "..."\n'
        "}\n"
        "반드시 JSON만 출력하고 다른 텍스트를 출력하지 마라. null 값은 JSON의 null이어야 한다."
    )

    messages = [{"role": "system", "content": system_prompt}]
    for turn in conversation:
        messages.append({"role": turn.get("role", "user"), "content": turn.get("content", "")})

    resp = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        max_tokens=800,
        temperature=0.0,
    )

    try:
        text = resp.choices[0].message.content
    except Exception:
        text = resp.choices[0].message.content if resp.choices else ""

    cleaned = text.strip()
    if cleaned.startswith("```"):
        cleaned = "\n".join(cleaned.splitlines()[1:-1]).strip()

    try:
        parsed = json.loads(cleaned)
        return parsed
    except Exception:
        template = {
            "purpose": None,
            "relation": None,
            "gender": None,
            "preferred_color": None,
            "personality": None,
            "has_dog": None,
            "has_cat": None,
            "user_experience": None,
            "preferred_style": None,
            "plant_type": None,
            "season": None,
            "humidity": None,
            "isAirCond": None,
            "watering_frequency": None,
            "emotion": None,
        }
        return template

def extract_partial_fields(initial_text: str) -> Dict[str, Any]:
    """
    LLM에게 initial_text를 보내서 initial_text에서 명확히 드러나는 필드들만 JSON으로 반환하게 함.
    반환된 키들만 채워서 caller에게 반환.
    """
    prompt = (
        "아래 문장을 보고, 해당 문장에서 명확히 드러나는 정보(가능한 항목들)만 JSON으로 반환하라. "
        "값이 문장에 명확히 드러나지 않으면 해당 키를 제외하라. "
        "출력 키: purpose, relation, gender, preferred_color, personality, has_dog, has_cat, user_experience, preferred_style, plant_type, season, humidity, isAirCond, watering_frequency, emotion\n"
        "문장:\n"
        f"{initial_text}\n\n"
        "예: '친구 졸업식에 선물하려고' -> {\"purpose\": \"친구 졸업식에 선물하려고\", \"relation\": \"친구\"}\n"
        "반드시 JSON만 출력하라."
    )

    messages = [{"role": "system", "content": prompt}]
    resp = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        max_tokens=400,
        temperature=0.0,
    )
    try:
        text = resp.choices[0].message.content
    except Exception:
        text = resp.choices[0].message.content if resp.choices else ""

    cleaned = text.strip()
    if cleaned.startswith("```"):
        cleaned = "\n".join(cleaned.splitlines()[1:-1]).strip()

    try:
        parsed = json.loads(cleaned)
        if "has_dog" in parsed or "has_cat" in parsed:
            pd = {}
            pd["has_dog"] = parsed.get("has_dog", None)
            pd["has_cat"] = parsed.get("has_cat", None)
            parsed.update(pd)
        return parsed
    except Exception:
        return {}

def save_json(data: Dict[str, Any], filename: str = "collected_user_info.json"):
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print(f"저장 완료: {filename}")

def main():
    print("안녕하세요 — 친절한 화원 직원입니다. 먼저 선물 상황을 간단히 알려주세요.")
    print("예: '친구 졸업식에 선물하려고' 같은 한두 문장으로 설명해 주세요.")
    print("답변을 원하지 않으면 'pass'를 입력하세요. 바로 추천(종료) 받고 싶으면 'end'를 입력하세요.")
    print("모르는 경우에는 '모름' 또는 '모르겠음' 등으로 답해 주세요.")
    print()

    collected: Dict[str, Optional[Any]] = {
        "purpose": None,
        "relation": None,
        "gender": None,
        "preferred_color": None,
        "personality": None,
        "has_dog": None,
        "has_cat": None,
        "user_experience": None,
        "preferred_style": None,
        "plant_type": None,
        "season": None,
        "humidity": None,
        "isAirCond": None,
        "watering_frequency": None,
        "emotion": None,
    }

    conversation: List[Dict[str, str]] = []

    # 1) 맨 처음: 전체적인 상황 설명 받기
    print(INITIAL_OPEN_QUESTION)
    initial = input("A: ").strip()
    conversation.append({"role": "user", "content": initial})

    if initial.strip().lower() == "end":
        print("지금까지의 대화를 분석해서 JSON으로 저장합니다...")
        extracted = extract_json_from_conversation(conversation)
        save_json(extracted)
        return
    if initial.strip().lower() == "pass":
        initial_parsed = {}
    elif is_unknown(initial):
        initial_parsed = {}
    else:
        initial_parsed = extract_partial_fields(initial)

    for k, v in initial_parsed.items():
        if k in collected:
            collected[k] = v
    if "has_pet" in initial_parsed:
        pet_map = parse_pets(initial_parsed.get("has_pet"))
        collected["has_dog"] = pet_map["has_dog"]
        collected["has_cat"] = pet_map["has_cat"]

    # 2) 남은 필드들만 질문
    for key, question in FIELDS:

        if key == "has_pet":
            continue

        if collected.get(key) is not None:
            continue

        context_summary = []

        if collected.get("relation"):
            context_summary.append(f"{collected.get('relation')}분")
        if collected.get("purpose"):
            context_summary.append(f"{collected.get('purpose')}")
        if context_summary:
            lead = " ".join(context_summary)
            print(f"Q: {lead} 관련해서, {question}")
        else:
            print(f"Q: {question}")

        user_input = input("A: ").strip()
        conversation.append({"role": "user", "content": user_input})

        low = user_input.strip().lower()
        if low == "end":
            print("지금까지의 대화를 분석해서 JSON으로 저장합니다...")
            extracted = extract_json_from_conversation(conversation)

            if "has_dog" not in extracted or "has_cat" not in extracted:
                pet_field = None
                for turn in reversed(conversation):
                    if any(k in turn["content"] for k in ["강아지", "고양이", "없음", "개", "고양"]):
                        pet_field = turn["content"]
                        break
                pet_parsed = parse_pets(pet_field) if pet_field else {"has_dog": None, "has_cat": None}
                extracted["has_dog"] = extracted.get("has_dog", pet_parsed["has_dog"])
                extracted["has_cat"] = extracted.get("has_cat", pet_parsed["has_cat"])
            save_json(extracted)
            return

        if low == "pass":
            print(f"'{key}' 항목은 건너뜁니다.")
            collected[key] = None
            continue

        if is_unknown(user_input):
            collected[key] = None
            continue

        if key == "gender":
            collected[key] = user_input
        elif key == "preferred_color":
            collected[key] = user_input
        elif key == "personality":
            collected[key] = user_input
        elif key == "user_experience":
            collected[key] = user_input
        elif key == "preferred_style":
            collected[key] = user_input
        elif key == "plant_type":
            collected[key] = user_input
        elif key == "season":
            collected[key] = user_input
        elif key == "humidity":
            collected[key] = user_input
        elif key == "isAirCond":
            collected[key] = user_input
        elif key == "watering_frequency":
            collected[key] = user_input
        elif key == "emotion":
            collected[key] = user_input
        else:
            collected[key] = user_input

        if any(k in user_input for k in ["강아지", "고양이", "개", "고양"]):
            pets = parse_pets(user_input)
            collected["has_dog"] = pets["has_dog"]
            collected["has_cat"] = pets["has_cat"]

    print("지금까지 수집된 내용을 바탕으로 JSON파일 저장")
    final_json = collected.copy()
    save_json(final_json)

if __name__ == "__main__":
    main()


안녕하세요 — 친절한 화원 직원입니다. 먼저 선물 상황을 간단히 알려주세요.
예: '친구 졸업식에 선물하려고' 같은 한두 문장으로 설명해 주세요.
답변을 원하지 않으면 'pass'를 입력하세요. 바로 추천(종료) 받고 싶으면 'end'를 입력하세요.
모르는 경우에는 '모름' 또는 '모르겠음' 등으로 답해 주세요.

선물하실 상황을 간단히 알려주세요. (예: 친구 졸업식 선물, 부모님 기념일 등) — 누구에게, 어떤 목적, 언제 주실 예정인지 한두 문장으로 설명해 주세요.
Q: 친척분 친척 졸업식에 선물하려고 관련해서, 선물받는 분의 성별은 무엇인가요? (여성 / 남성 / 기타)
Q: 친척분 친척 졸업식에 선물하려고 관련해서, 원하시는 색상을 입력해주세요.


In [ ]:
class Modelcollect:
	def __init__(self, tools):        
        self.tools = tools

	def get_response(self, tools):
		
        prompt = """
        
		너는 식물에 대해 차분하게 상담해 주는 전문가이다.
        아래 형식을 반드시 지키되, 실제 상담사가 말하듯 자연스럽고 단정적인 말투로 작성한다.
		
		### 사용자 입력 특수키워드 처리 ###
        - 사용자가 "모름", "모르겠음", "모르겠어요", "몰라" 등으로 응답하면 해당 항목의 값은 null 로 처리한다.
        - 사용자가 "pass"라고 입력하면 그 질문은 건너뛴다(값은 null).
        - 사용자가 "end"라고 입력하면 대화를 더 진행하지 말고 지금까지의 대화 전체를 분석해서 아래에 지정한 JSON 스키마 형식으로만 출력하라.
		
		### 수집 항목 ###
        - 필수 수집 항목 : purpose, relation, gender, preferred_color, personality, has_dog, has_cat
		- 추가 수집 항목 : user_experience, preferred_style, plant_type, season, humidity, isAirCond, watering_frequency, emotion		
		
		### 출력 형식 ###
		- 평상시(사용자가 end를 **아직** 입력하지 않은 경우): 자연스러운 한국어 상담 응답을 한다(질문, 안내, 확인 등).
        - 사용자가 "end"를 입력한 경우: **반드시** 아래 JSON 스키마만 출력하라. JSON 키 이름과 타입을 정확히 지켜야 한다. 값이 없으면 `null` 로 둔다(문자열 "null" X).
        """
		
        system_msg = SystemMessage(prompt)
        input_msg = [system_msg] + messages

        model = ChatOpenAI(
            model="gpt-4o-mini", 
            temperature=0.3
        ).bind_tools(self.tools)

        response = model.invoke(input_msg)
        
        return response

TabError: inconsistent use of tabs and spaces in indentation (<string>, line 3)

In [ ]:
@tool